# Homework 3 of group 12 with Flaminia Matteucci, Muhammad Anwar Alam, Michael Aichmueller


In [1]:
from SearchEngine import SearchEngine

The search engine was build into its own class called *SearchEngine* within its likewise named file *SearchEngine.py*. The basics of this class follow a simple procedure:
- When instantiating the class, the \__init__ function will compute certain facts:
    - Among these facts are the maximum rate per night, the most recent date of listing, number of document, all cities ever named, and the maximum number of bedrooms. Something that would need to be stored in a file and reupdated if this was ever to be made into a more serious search engine.
    - It will also initialize a check of the essential NLTK files being installed and instantiate stemmer and stopwords.
- The crucial boolean parameter *build_essentials* decides whether to immediately begin with creating the vocabulary (or reading it), building the IDFs, and building the inverted index. If so, all of this is done with a call to the method *\_build_invert_idx* with the parameter of docs (the document type)

Generally, this class loads data either by the method *\_load_data_complete*, which loads the data file and subsequently cleans it, or by *\_load_data_by_nr*, which loads the given documents by their number from the split document files.

Note, that the class only works if the data file *Airbnb_Texas_Rentals.csv* is placed in the data_dir subdirectory (by default "./data/") !

In [2]:
engine = SearchEngine(build_essentials=True)

'_build_invert_idx'  10.44 s


The class assumes the AirBnB file has already been split. If not, one needs to call (otherwise it breaks when trying to search) 

**SearchEngine.split_data_in_ads()**

which will load the AirBnB data and split it. The files are normally put into the *self.processed* folder, which is a subdirectory of the *self.data* folder (both changeable as parameters of the *\__init__* method.

Note, that we opted to remove all duplicate entries in this data file, meaning only the first of the duplicates is kept (of which there are many: 18k to 11k files after duplicates removal). Duplicates are recognized by equality among title, description and city.

In [3]:
engine.split_data_in_ads()

Task 3.1
--------------
For task 3.1 of homework 3 of ADM, the basic search engine, the specific code is found in *SearchEngine._process_query_rel_docs*. This function can take the bool parameter *conjunctive* which, if TRUE (default), will search for documents containing all the words of the query. If FALSE, it will find all the documents containing any of the words of the query. The search initiated by a call to the method *search_conjunctive* which in turn calls the aforementioned method with *conjunctive=True*.

In [4]:
docs = engine.search_conjunctive()

Please enter search query:  beautiful house with garden
searching...
'_process_query_rel_docs'  0.07 s
Search finished.

Search query: beautiful house with garden
Seach results:

Doc-Nr|       Title       |        Description         |     City     |          Url          
  1   |  Unique Location! |  Stylish, fully remodeled  |  San Antonio | https://www.airbnb.co 
      |   Alamo Heights - |    home in upscale NW –    |              | rooms/17481455?locati 
      |      Designer     |     Alamo Heights Area.    |              |    =Cibolo%2C%20TX    
      |      Inspired     |  Amazing location - House  |              |                       
      |                   |   conveniently located...  |              |                       
 9348 |    Stylish Stone  |      Newly remodeled,      |    Dallas    | https://www.airbnb.co 
      |  Cottage on White |     open-concept living    |              | rooms/15132655?locati 
      |      Rock Lake    |   located one block from   |     

 9175 |   Dallas Downton  |  Our Dallas Downton Abbey  |    Dallas    | https://www.airbnb.co 
      |        Abbey      |    is close to downtown,   |              | rooms/14254297?locati 
      |                   |     Southern Methodist     |              |    =Addison%2C%20TX   
      |                   |    University, George W.   |              |                       
      |                   |      Bush Library, ...     |              |                       
 8033 |  Historic Mansion |   Less than 2 miles from   |  Fort Worth  | https://www.airbnb.co 
      |     Downtown Ft   |     TCU, the Fort Worth    |              | rooms/8664808?locatio 
      |     Worth; Bass   |      Convention Center     |              |    Benbrook%2C%20TX   
      |      Hall; TCU    |   Botanical Gardens; Bass  |              |                       
      |                   |    Hall, Magnolia Av...    |              |                       
 3445 |       Vintage     |   This fantastic backy

# Task 3.2

The cosine search uses the engine 3.1 to do a conjunctive search within the documents first to find all the relevant documents. Afterwards it computes the cosine similarity using sparse matrices from scipy and the cosine similarity function from scikit, which can handle sparse matrices.

In [5]:
docs = engine.search_cosine(top_k=10)

Please enter search query:  beautiful house with garden
searching...
'_process_query_rel_docs'  0.07 s
'_process_query_cosinesim'  0.24 s
Search finished.

Search query: beautiful house with garden
Seach results:

Doc-Nr|      Title       |        Description        |     City    |         Url          | Score 
 6163|    Stay at an    |     A beautiful, modern   |    Austin   | https://www.airbnb.c | 0.325 
     |    architect's   |    home, surrounded by a  |             | /rooms/18998883?loca |       
     |   gorgeous home  |    beautiful garden and   |             | on=Cedar%20Park%2C%2 |       
     |    and garden!   |       full of natural     |             |          X           |       
     |                  |    light!Designed by...   |             |                      |       
 7244|  Greeley Gardens |  This is a beautiful cozy |    Houston  | https://www.airbnb.c | 0.268 
     |  Carriage House  |   garage apartment with a |             | /rooms/8301898?locat |       
 

# Task 3.3

The custom score is a weighted score [0,1] with the following 4 categories being relevant:
- distance
- rate per night
- recentness
- cosine similarity (utilizing 3.2)

All four of these factors are weighted (chosen to be fixed weights) by 0.4, 0.3, 0.2, 0.1 respectively. The necessary information for the computation of the scale is to be entered in the prompted input bar.

The search uses a (non) conjunctive search of part 3.1 first, in order to find the relevant documents, and then starts computing the score on these. Technical details of the search can be found in the method *\_process_query_happy_score*.

In [6]:
docs = engine.search_happy_score(top_k=10)

Please enter search query:  beautiful house with garden
Optionally you may provide the following information:

Cities (space separated): Houston Austin
Maximum rate per night: 100
Minimum number of bedrooms: 1
Date of listing after (MM/YY):  01/15
Search initialized...
'_process_query_rel_docs'  8.65 s
'_process_query_happy_score'  15.03 s
Search finished.

Search query: beautiful house with garden
Seach results:

Doc-Nr|      Title       |        Description        |     City    |         Url          | Score 
 9274|  1.1) Hostel 1mi |  1) CONVENIENTLY LOCATED, |    Houston  | https://www.airbnb.c | 0.722 
     |  from Downtown & |    CLOSE to the ASTROS,   |             | /rooms/16583494?loca |       
     |     Metrorail    |      ROCKETS, DYNAMOS,    |             |  on=Baytown%2C%20TX  |       
     |                  |     CONVENTION CENTER,    |             |                      |       
     |                  |       THEATERS, BA...     |             |                      | 

# Task 4

The folium map is supposed to contain a center point and a radius around it that selects the ads within this zone and highlights them through markers. This is done by the method *create_map* which creates the file *houses_within_radius.html*, which can be loaded and inspected from the repository.

In [7]:
engine.create_map()

Choose coordinates (reverting to default if empty string given):
Latitude: 
Longitude: 
Choose radius: 10
'create_map'  9.77 s
